In [1]:
import sqlite3
import datetime
conn = sqlite3.connect('/home/arcyleung/Projects/frigate/config/frigate.db')
cursor = conn.cursor()

print("Tables:")
for row in cursor.execute("SELECT name FROM sqlite_master WHERE type='table';"):
    print(row[0])

# Define the start time (now - 24 hours)
end_time = datetime.datetime.now()
start_time = end_time - datetime.timedelta(hours=24)

print("schema:", cursor.execute("SELECT * FROM event").description)

# Read the "event" table and filter records with "start_time" within the last 24 hours
cursor.execute("SELECT * FROM event WHERE start_time >= :start_time AND start_time <= :end_time", {'start_time': start_time.timestamp(), 'end_time': end_time.timestamp()})
events = cursor.fetchall()

# Print the filtered events
print(len(events), "events found")
for event in events:
    print(event[0], event[1])

conn.close()


Tables:
migratehistory
recordings
timeline
event
regions
schema: (('id', None, None, None, None, None, None), ('label', None, None, None, None, None, None), ('camera', None, None, None, None, None, None), ('start_time', None, None, None, None, None, None), ('end_time', None, None, None, None, None, None), ('top_score', None, None, None, None, None, None), ('false_positive', None, None, None, None, None, None), ('zones', None, None, None, None, None, None), ('thumbnail', None, None, None, None, None, None), ('has_clip', None, None, None, None, None, None), ('has_snapshot', None, None, None, None, None, None), ('region', None, None, None, None, None, None), ('box', None, None, None, None, None, None), ('area', None, None, None, None, None, None), ('retain_indefinitely', None, None, None, None, None, None), ('sub_label', None, None, None, None, None, None), ('ratio', None, None, None, None, None, None), ('plus_id', None, None, None, None, None, None), ('score', None, None, None, None, Non

In [3]:
import os

def get_filenames(path):
    filenames = []
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith('-clean.png'):
                filenames.append(os.path.join(root, file))
    return filenames

# filenames = get_filenames('/tmp/storage/clips')
# print(filenames)

In [4]:
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
import torch
from PIL import Image
import requests

processor = LlavaNextProcessor.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf")

model = LlavaNextForConditionalGeneration.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf", torch_dtype=torch.float16, low_cpu_mem_usage=True, load_in_4bit=True, use_flash_attention_2=False)
# model.to("cuda:0")



/home/arcyleung/Projects/cantorfs/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Loading checkpoint shards: 100%|██████████| 4/4 [00:10<00:00,  2.70s/it]


In [15]:


# Define a chat histiry and use `apply_chat_template` to get correctly formatted prompt
# Each value in "content" has to be a list of dicts with types ("text", "image")
conversation = [
    {

      "role": "user",
      "content": [
          {"type": "text", "text": "Describe any people in the scene, what clothing they are wearing, orientation with other objects, anything they are holding and what do they appear to be doing. Also describe prominent vehicles or legible text, and do not mention anything that shoule be ordinary."},
          {"type": "image"},
        ],
    },
]
prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)


In [22]:

from PIL import Image
import datetime
import json

def transcribe_one(filename, responses):
    image = Image.open(filename)
    inputs = processor(prompt, image, return_tensors="pt").to("cuda:0")

    # autoregressively complete prompt
    output = model.generate(**inputs, max_new_tokens=150)

    text = processor.decode(output[0], skip_special_tokens=True)
    responses[filename] = text[len(prompt) - 5:]

responses = {}
today = datetime.datetime.now().isoformat()

# for idx, event in enumerate(events):
#     filename = f"/tmp/storage/clips/front-{event[0]}-clean.png"
#     print(f"{idx} of {len(events)}")
#     transcribe_one(filename, responses)

transcribe_one("/tmp/storage/clips/front-1722009392.383259-1jcmy8-clean.png", responses)

print(responses)

json.dump(responses, open(f"analyze_{today}.json", "w"))


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'/tmp/storage/clips/front-1722002238.389493-rlqwm8-clean.png': 'In the image, there are two individuals visible. One person is standing on the sidewalk, wearing a red jacket and a white cap, and appears to be looking towards the camera. The other person is seated in a red car, which is parked on the street. The car is facing towards the right side of the image.\n\nProminent vehicles in the scene include a white van parked on the street and a black car parked in a driveway. There is no legible text visible in the image.\n\nThe orientation of the objects is such that the red car is in the foreground, with the white van and the black car positioned behind it. The person standing on the sidewalk is to the left of the red car, and the person in the red car is to the right of the white van. The person in the red car is holding a cell phone, and it appears they are either using the phone or preparing to use it. '}
